In [1]:
%load_ext autoreload
%autoreload 2

import pdfplumber
import re

from utils import create_new_sheet, write_content, format_sheet

import os
token_filepath = '../google_token.json'
if os.path.exists(token_filepath):
    os.remove(token_filepath)

In [2]:
############ Update the following parameters ##################
FILEPATH = "../data/June statement.pdf"
NEW_SHEET_NAME = "June 2025"

In [3]:
SPREADSHEET_ID = "1Ik2lyOO5MN9a8-hEUf4sRqUlP3WcN36ItekQSGV4NxY"

In [4]:
pdf_lines = []
with pdfplumber.open(FILEPATH) as pdf:
    for page_id in range(len(pdf.pages)):
        page = pdf.pages[page_id]
        pdf_lines.extend(page.extract_text_lines())
pdf_lines = [line['text'] for line in pdf_lines]

In [5]:
import os
print("PYTHONPATH:", os.environ.get('PYTHONPATH'))
print("PATH:", os.environ.get('PATH'))

PYTHONPATH: None
PATH: /Users/shashankverma/Documents/Stuff/miniconda/envs/llm1/bin:/Users/shashankverma/.nvm/versions/node/v23.11.0/bin:/opt/homebrew/opt/node@20/bin:/opt/homebrew/opt/node@20/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/Users/shashankverma/Documents/Stuff/miniconda/envs/llm1/bin:/Users/shashankverma/Documents/Stuff/miniconda/condabin:/usr/local/git/git-google/bin:/usr/local/git/current/bin:/usr/local/bin:/usr/bin:/bin:/usr/local/sbin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin


In [6]:
import pickle
# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# lr_model = Pipeline([('vect', CountVectorizer()),
#                ('tfidf', TfidfTransformer()),
#                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
#               ])

with open('pickle/lr_model.pkl', 'rb') as f:
    lr_model = pickle.load(f)
# lr_model = pickle.load(open('pickle/lr_model.pkl', 'rb'))

/Users/shashankverma/Documents/Stuff/miniconda/envs/llm1/lib/python3.11/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator CountVectorizer from version 1.4.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/shashankverma/Documents/Stuff/miniconda/envs/llm1/lib/python3.11/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.4.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/shashankverma/Documents/Stuff/miniconda/envs/llm1/lib/python3.11/site-packages/sklearn/base.py:380: Inconsistent

In [7]:
id_to_label = {0: 'Regular', 1: 'Trip', 2: 'Hobby'}

In [8]:
regex_pattern = "^(\d\d/\d\d.*) (-*[\d\.,]+)$"
accumulated_content = []
for line in pdf_lines:
    if "AUTOMATIC PAYMENT" in line:
        continue
    re_search = re.search(regex_pattern, line)
    if re_search is not None:
        transaction_name = re_search.group(1)
        expense_type_id = lr_model.predict([transaction_name])[0]
        expense_type = id_to_label[expense_type_id] if expense_type_id in id_to_label else 'Regular'
        cost = float(re_search.group(2).replace(',', ''))
        accumulated_content.append([transaction_name, cost, expense_type])

In [9]:
# Sort the content according to the transaction date
accumulated_content = sorted(accumulated_content, key=lambda x: x[0].split(' ')[0])

for i in range(len(accumulated_content)):
    row_id = i + 2
    regular_cost = f'=IF(C{row_id}="Regular", B{row_id}, 0)'
    trip_cost = f'=IF(C{row_id}="Trip", B{row_id}, 0)'
    hobby_cost = f'=IF(C{row_id}="Hobby", B{row_id}, 0)'
    accumulated_content[i].append(regular_cost)
    accumulated_content[i].append(trip_cost)
    accumulated_content[i].append(hobby_cost)

accumulated_content = [['Transaction', 'Cost', 'Type', 'Regular expenses', 'Trip expenses', 'Hobby expenses']] + accumulated_content + [['', f'=SUM(B2:B{row_id})', '', f'=SUM(D2:D{row_id})', f'=SUM(E2:E{row_id})', f'=SUM(F2:F{row_id})', f'=SUM(D{row_id+1}:F{row_id+1})']]

In [13]:
# Export to the spreadsheet.
create_new_sheet_response = create_new_sheet(SPREADSHEET_ID, NEW_SHEET_NAME)
sheet_id = create_new_sheet_response['replies'][0]['addSheet']['properties']['sheetId']
write_response = write_content(SPREADSHEET_ID, NEW_SHEET_NAME, accumulated_content)
format_sheet_response = format_sheet(SPREADSHEET_ID, sheet_id)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=694301136407-7lfplojamr3flfdqoo629e5mf15pv8s7.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A57642%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=CO40lrDOZkxKLHxZia0KBJXwq7rxCD&access_type=offline
Created a new sheet result: {'spreadsheetId': '1Ik2lyOO5MN9a8-hEUf4sRqUlP3WcN36ItekQSGV4NxY', 'replies': [{'addSheet': {'properties': {'sheetId': 1790560409, 'title': 'June 2025', 'index': 24, 'sheetType': 'GRID', 'gridProperties': {'rowCount': 1000, 'columnCount': 26}}}}]}
Exported content to 427 cells.
Format sheet result: {'spreadsheetId': '1Ik2lyOO5MN9a8-hEUf4sRqUlP3WcN36ItekQSGV4NxY', 'replies': [{}, {}, {}]}
